<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI285 - Computación Científica I / INF285 - Computación Científica</h1>
    <h1> Tarea 3:  </h1>
    <h3> [S]cientific [C]omputing [T]eam 2019</h3>
</center>
<p>
<center>Junio 2019 - v2.0 </center>
</p>

---

## Contexto

Hasta hace no más de una década, los dibujos animados y videojuegos eran visualizados con un *framerate* de entre 12 a 24 *frames* por segundo (*FPS*). Sin embargo, las mejoras en las tecnologías nos permiten hoy disfrutar de videos con una mucho mejor fluidez, siendo el *framerate* estándar actual de 60 *FPS*.

### ¿Qué es Interpolación de Movimiento o *Motion-Compensated Frame Interpolation* (MCFI)? 

Corresponde a un método de **procesamiento de video** en el cual se generan *frames* de animación entre los *frames* existentes, utilizando métodos de interpolación. El objetivo principal es mejorar la fluidez del video.

En esta tarea implementaremos MCFI aplicando los algoritmos de interpolación aprendidos en la clase de Computación Científica, para mejorar el *framerate* de un video que posee $N$ FPS (*Frames Per Second*), para luego generar un nuevo video con un total de *frames* mayor al original.

---

#### Librerías

In [1]:
from __future__ import print_function
import numpy as np
#pip3 install imutils
import imutils
#pip3 install opencv-python
import cv2 
import matplotlib.pyplot as plt
import scipy.interpolate as scp
import base64
from IPython.display import HTML

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

---

## Problema

El problema, descrito de forma general, corresponde al incremento de los FPS de un video, esperando mejorar la fluidez de este. Junto a esta tarea, se adjuntan cuatro videos. El nombre de cada video sigue el formato `video_Nfps.mp4` [1], donde $N$ indica la cantidad de FPS del video. Para esta tarea, trabajaremos con videos con 60, 30, 15 y 5 FPS. Una vez incrementados los FPS, compararemos los resultados con la versión *original* del video.

Para ilustrar de manera inicial lo que se desea realizar, considere un video compuesto por los 2 siguientes *frames*:

<img src="interpolacion2.png" style="float:center;height:200px">

Su algoritmo de interpolación debe ser capaz de construir una o más funciones, utilizando como *data* los frames disponibles del video. En este ejemplo, podríamos construir una interpolación utilizando como datos los dos *frames* disponibles y, mediante esta función, se pueden generar uno o más *frames* dentro de la secuencia para aumentar el *framerate* de su video:

<img src="interpolacion3.png" style="float:center;height:198px">

---

## Sección 1 (40 puntos). Testing de métodos de interpolación

En esta pregunta implementaremos una forma sencilla de interpolación de videos. Considere que el video contiene un total de $M$ frames.

El proceso de interpolación será **a través de cada pixel**, es decir, si se desea interpolar el pixel $(i, j)$ y generar un nuevo frame, entonces el interpolador se debe construir utilizando como *data* todos los pixeles $(i, j)$ del video. De una forma más matemática, el conjunto de datos está definido por:

$$
    S_{i,j} = \{ (x_1, I_1[i, j]), (x_2, I_2[i,j]), \dots, (x_M, I_M[i,j]) \},
$$

donde $x_i$ corresponde a un punto siguiendo una distribución equiespaciada o de Chebyshev e $I_k[i,j]$ denota el valor del pixel $(i,j)$ del $k$-ésimo frame. Su labor en esta pregunta es interpolar los datos de $S_{i,j}$ y así construir los pixeles que se desean agregar entre cada par de frames.

1. Implemente la función _interpolate_\__frames (frames, dst_\__fps, seconds, interpolator)_. Esta función recibe como parámetros:
    * La matriz de _frames_: Video original a interpolar.
    * La cantidad original de *fps*.
    * La cantidad de _fps_ deseados (mayor a los _fps_ originales).
    * La cantidad de segundos que tendrá el video.
    * Un string _interpolator_, cuyos valores posibles son:
        * "spline-cubica": Spline cúbica. Estudie y seleccione un tipo de condición de borde adecuado para el problema, justificando su elección.
        * "spline-lineal": Spline lineal.
        * "lagrange": Interpolación polinomial mediante el método Interpolación de Lagrange.
        * "baricentrica": Interpolación Baricéntrica [2].
    * Un string 'points_type', cuyos valores posibles son:
        * "equi": Para usar una distribución equiespaciada de puntos en el eje de las abscisas.
        * "cheb": Los x_i (o eje de las abscisas) son los puntos de Chebyshev.

    En caso de utilizar código ajeno, ya sean librerías, notebooks del curso u otro, recuerde referenciar apropiadamente.<br/><br/>

2. Mediante la función implementada anteriormente, aumentaremos los *framerates* de los videos disponibles. Para esto:
    * Tomando como base el _video_\__30fps.mp4_, genere un video de 60 fps.
    * Tomando como base el _video_\__15fps.mp4_, genere un video de 30 fps.
    * Tomando como base el _video_\__5fps.mp4_, genere un video de 15 fps.
    * Tomando como base el _video_\__5fps.mp4_, genere un video de 60 fps.

    Para esto, utilize los 4 interpoladores generados y las distribuciones de puntos especificadas. Mida los tiempos de cómputos de cada método, con cada distribución de puntos.

In [ ]:
'''
frames: array of frames (height x width x total_frames)
old_fps: Number of fps of the original video
new_fps: fps of output matrix 
seconds: seconds of video 
interpolator: Interpolator type
points_type: Points distribution to use

return: dst_frames (array of frames (height x width x dst_fps*seconds))
'''

def interpolate_frames(frames, old_fps, new_fps, seconds, interpolator, points_type):

    x = np.arange(0, frames.shape[2], 1)
    a = np.linspace(0, frames.shape[2]-1, num=frames.shape[2]*2-1)
            
    height = frames.shape[0]
    width = frames.shape[1]
 
    #dst_frames = np.zeros((height, width, round(new_fps)*seconds), dtype=np.uint8)
    dst_frames = np.zeros((height, width, round(new_fps)*seconds), dtype=np.int64)
    
    # Your code goes here!
    if(new_fps > old_fps):
        if(interpolator == "spline-cubica"):
            print("spline-cubica")
                 
            for i in range(height):
                for j in range(width):
                    ys = frames[i, j, :]
                    Sp = cubic_spline(x, ys, 'Natural')
            
                    for k in range(frames.shape[2]-1):
                        x1 = np.linspace(k, k+1, 3)
                        S = ys[k]+Sp['b'][k]*(x1-x[k])+Sp['c'][k]*(x1-x[k])**2+Sp['d'][k]*(x1-x[k])**3
                
                        dst_frames[i][j][k] = S[0]
                        dst_frames[i][j][k+1] = S[1]
                
                        if(k == frames.shape[2]-2):
                            dst_frames[i][j][k+1] = S[2]
                   
            fps = int(dst_frames.shape[2]/seconds)
            print(frames)
            print(dst_frames)
            new_video(dst_frames, fps, width, height)

        elif(interpolator == "spline-lineal"):
            print("spline-lineal")
        elif(interpolator == "lagrange"):
            print("lagrange")
                        
            for i in range(height):
                for j in range(width):
                    ys = frames[i, j, :]
                    
                    D = Lagrange(x, ys)
                    A = scp.lagrange(x, ys)
                    print(A(0))
                
                    for k in range(len(a)):
                        dst_frames[i][j][k] = A(a[k])
                   
            fps = int(dst_frames.shape[2]/seconds)
            print(frames)
            print(dst_frames)
            #new_video(dst_frames, fps, width, height)
            
        elif(interpolator == "baricentrica"):
            print("baricentrica")
        else:
            print("Interpolador inválido")
    return dst_frames
    
def cubic_spline(x, y, end=None, k1=0, k2=0, p1=0, p2=0):
    #x: x-coordinates of points
    #y: y-coordinates of points
    #end: Natural, Adjusted, Clamped, Parabolically, NaK
    
    n = len(x)
    A = np.zeros((3*n-3, 3*n-3))
    b = np.zeros(3*n-3)
    
    delta_x=np.diff(x)
       
    #Building the linear system of equations
    
    #1st property
    for i in np.arange(n-1):
        b[i]= y[i+1]-y[i]
        A[i,3*i:3*(i+1)] = [delta_x[i],delta_x[i]**2,delta_x[i]**3]
    #2nd property
    for i in np.arange(n-2):
        A[(n-1)+i,3*i:3*(i+1)+1]=[1, 2*delta_x[i], 3*delta_x[i]**2, -1]
    #3rd property
    for i in np.arange(n-2):
        A[(n-1)+(n-2)+i,3*i:3*(i+1)+2] = [0, 2, 6*delta_x[i], 0, -2]
    
    #Ending conditions (4th property)
    if end =='Natural':
        A[-2,1]= 2
        A[-1,-2] = 2
        A[-1,-1] = 6*delta_x[-1]

    elif end == 'Adjusted':
        A[-2,1]= 2
        A[-1,-2] = 2
        A[-1,-1] = 6*delta_x[-1]
        b[-2:] = [k1,k2]
        print('Adjusted',b[-2:])

    elif end == 'Clamped':
        A[-2,0]=1
        A[-1,-3:] = [1,2*delta_x[-1],3*delta_x[-1]**2]
        b[-2:] = [p1,p2]

    elif end == 'Parabolically':
        A[-2,2]=1
        A[-1,-1]=1

    elif end == 'NaK':
        A[-2,2:6]=[6,0,0,-6]
        A[-1,-4:]=[6,0,0,-6]
    
    #Solving the system
    sol = np.linalg.solve(A,b)
    S = {'b':sol[::3],
         'c':sol[1::3],
         'd':sol[2::3],
         'x':x,
         'y':y
        }
    return S

def Lagrange(x, y, show=False):
    # We calculate the li's 
    p = np.array([y[i]/np.prod(x[i] - np.delete(x,i)) for i in range(len(x))])
    
    # The function shows the data if the flag is true
    if show:
        print('Data Points: '); print([(x[i],y[i]) for i in range(len(x))])
        xS = sp.Symbol('x')
        L = np.dot(np.array([np.prod(xS - np.delete(x,i))/np.prod(x[i] - np.delete(x,i)) for i in range(len(x))]),y)
        print('Interpolation Function: '); 
        print(L)
        
    # Finally, we return a data structure with our interpolating polynomial        
    D = {'M':'Lagrange',
         'P':p,
         'x':x,
         'y':y}
    return D
    
frames = video_to_matrix('video_30fps.mp4', 1)
interpolate_frames(frames, 30, 60, 1, "lagrange", "equi")

Para simplificar el manejo del video en el *notebook*, se les facilita la función `video_to_matrix` que les permitirá abrir un video y transformarlo a una estructura de datos de *numpy*. El resultado será una *array* de dimensión $\text{height}\times\text{width}\times\text{Total de frames}$, donde *height* corresponde a la altura (en pixeles) del video y *width* al ancho del video.

In [3]:
'''
Carga de video a matriz de [height, width, fps*seconds], dejando en escala de grises cada frame, 
reduciendo su dimensionalidad.

Input:
    src_name: nombre (o path) de video de origen
    seconds: cantidad de tiempo en segundos a considerar del video de origen.

Output: 
    frames: matriz de shape [height, width, fps*seconds], la cual contiene los frames del video original
    number_fps: frames por segundo del video original
'''
def video_to_matrix(src_name, seconds):
    # Se lee el video
    stream = cv2.VideoCapture(src_name)
    
    # Dimension del video y fps
    height = round(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = round(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
    number_fps = stream.get(cv2.CAP_PROP_FPS)
    
    frames = np.zeros((height, width, round(number_fps)*seconds), dtype=np.uint8)
    
    for i in range(round(number_fps)*seconds):
        # Leer el video hasta que no hayan más datos
        (grabbed, frame) = stream.read()
        if not grabbed:
            break
            
        # Leer el frame y cambiarlo a escala de grises
        frame = cv2.resize(frame, (width, height)) 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames[:,:,i] = frame
        
    return frames

Además, el siguiente código sirve para visualizar un video dentro del mismo *notebook*.

In [25]:
HTML('\
<div align="middle">\
<video width="80%" controls>\
      <source src="./{0}" type="video/mp4">\
</video></div>'.format('video_60fps.mp4'))

## Sección 2 (15 puntos). Visualización del interpolador y del error, a través del tiempo

1. Utilizando el *widget* adjunto, visualice los resultados generados por los interpoladores, su error y comente al respecto. ¿Qué método le parece que tiene un mejor desempeño?

**Nota**: usted debe modificar el *widget* en las secciones necesarias para que lea correctamente los videos generados por usted, acorde a los parámetros que se pueden variar. Puede modificar secciones del *widget* para que se adapte a sus códigos.

In [ ]:
# Widget to plot the interpolation function
def plot_interpolation(method, points_dist, i, j, fps_increase_type):
    
    # Elegimos el video a trabajar
    if fps_increase_type == "5->15":
        frames = video_to_matrix('video_15fps.mp4', 5)
        interpolated_video_name = 'video_15fps.mp4' # Acá va el video de 15 fps que generó a partir del de 5 fps
    elif fps_increase_type == "5->60":
        frames = video_to_matrix('video_60fps.mp4', 5)
        interpolated_video_name = 'video_60fps.mp4' # Acá va el video de 60 fps que generó a partir del de 5 fps
    elif fps_increase_type == "15->30":
        frames = video_to_matrix('video_30fps.mp4', 5)
        interpolated_video_name = 'video_30fps.mp4' # Acá va el video de 30 fps que generó a partir del de 15 fps
    else:
        frames = video_to_matrix('video_60fps.mp4', 5)
        interpolated_video_name = 'video_60fps.mp4' # Acá va el video de 60 fps que generó a partir del de 30 fps
        
    # Un punto por frame
    x = np.arange(0, frames.shape[2], 1)
    
    # Usar el interpolador acá (Matrix aleatoria es simplemente como un ejemplo para introducir ruido)
    interp_frames = video_to_matrix(interpolated_video_name, 5) + np.random.rand(frames.shape[0], frames.shape[1], frames.shape[2])
    
    plt.figure(figsize=(15, 14))
    
    # Grafico del interpolador y los pixeles reales
    plt.subplot(211)
    plt.plot(x, interp_frames[i,j,:], 'rx-', label='Video interpolados')
    plt.plot(x, frames[i,j,:], 'bd', label='Video esperado')
    plt.ylim([0, 256])
    plt.title('Comparación entre los pixeles interpolados y los deseados')
    plt.xlabel('Frame')
    plt.ylabel('Valor pixel')
    plt.grid(True)
    
    # Grafico del error
    plt.subplot(212)
    error_matrix = frames - interp_frames
    plt.plot(x, error_matrix[i,j,:], 'sk-', label='Error de interpolación')
    mean_error = np.zeros(error_matrix.shape[2])
    sigma = mean_error.copy()
    for k in range(error_matrix.shape[2]):
        mean_error[k] = error_matrix[:,:,k].mean()
        sigma[k] = error_matrix[:,:,k].std()
    plt.plot(x, mean_error, '.-', label='Error medio de interpolación')
    plt.plot(x, mean_error + sigma, label='Error medio + desviación estándar')
    plt.plot(x, mean_error - sigma, label='Error medio - desviación estándar')
    plt.title('Error de interpolación')
    plt.xlabel('Frame')
    plt.ylabel('Error')
    plt.grid(True)
    plt.legend(loc='best')
    
    plt.show()

interact(
    plot_interpolation, 
    method=["spline-cubica","spline-lineal","lagrange","baricentrica"],
    points_dist=["equi","cheb"],
    j=(0, 511),
    i=(0, 269),
    fps_increase_type=["5->60", "5->15", "15->30", "30->60"]
);

## Sección 3 (25 puntos). Comparación de Videos

1. Utilizando los videos generados en la pregunta anterior, compare visualmente los videos generados con su versión de "origen" (video original al que se le agregaron nuevos *frames*) y con la versión "esperada" del video; por ejemplo, para la interpolación de 15fps a 30fps, el video de origen sería _video_\__15fps.mp4_, y el video esperado _video_\__30fps.mp4_. Comente al respecto. ¿Se realiza correctamente la interpolación? ¿Qué es lo que está haciendo visualmente la interpolación en los videos?

    Para ayudar a la visualización de los videos, puede hacer uso de la función _concat_\__2_\__videos (src_\__name, dst_\__name, seconds)_, la cual permite concatenar o "unir" dos videos con el mismo framerate para realizar una comparativa en un nuevo video generado.

    _NOTA: Debe hacer un preprocesamiento sobre el video de origen para compararlo con la interpolación, pues la función solo permite unir 2 videos de igual framerate. Una solución simple es repetir cada frame la cantidad necesaria para completar el framerate deseado_ <br/><br/>

2. Comente respecto a los tiempos de cómputo obtenidos. ¿Existe alguna correlación entre la calidad del video generado y los tiempos de cómputo?

3. Comente respecto al efecto de la distribución de puntos. ¿Genera alguna diferencia la distribución de puntos equiespaciada respecto a los puntos de Chebyshev?

In [ ]:
'''
src_name: name/path of first video
dst_name: name/path of second video
seconds : seconds of video to consider.

returns: None
'''

def concat_2_videos(src_name, dst_name, seconds):
    stream = cv2.VideoCapture(src_name)
    src_frames = video_to_matrix(src_name, seconds)
    print("FPS VIDEO "+src_name+": ",int(src_frames.shape[2]/seconds))
    
    stream_2 = cv2.VideoCapture(dst_name)  
    dst_frames = video_to_matrix(dst_name, seconds)
    print("FPS VIDEO "+dst_name+": ",int(dst_frames.shape[2]/seconds))

    # Close original videos
    stream.release()
    stream_2.release()
    
    if dst_frames.shape[2] != src_frames.shape[2]:
        print("No es posible concatenar dos videos de distinto fps")
        return
    
    fps = int(dst_frames.shape[2]/seconds)
    
    #concatenate both videos
    conc_frames = np.concatenate((src_frames, dst_frames), axis=1)
    
    height = conc_frames.shape[0]
    width = conc_frames.shape[1]
    
    #saving comparison on a new video
    out = cv2.VideoWriter("comparison_"+src_name+"_vs_"+dst_name,cv2.VideoWriter_fourcc('m','p','4','v'), fps, (width,height))
   
    for i in range(conc_frames.shape[2]):
        frame = np.uint8(conc_frames[:,:,i])
        out.write(np.dstack([frame, frame, frame]))
    
    out.release()
    cv2.destroyAllWindows() 

In [6]:
def new_video(frames, fps, width, height):
    out = cv2.VideoWriter("new_video.mp4",cv2.VideoWriter_fourcc('m','p','4','v'), fps, (width,height))
    
    for i in range(frames.shape[2]):
        frame = np.uint8(frames[:,:,i])
        out.write(np.dstack([frame, frame, frame]))
    
    out.release()
    cv2.destroyAllWindows()

## Sección 4 (10 puntos): Conclusión

Considerando todo el trabajo anterior, concluya al respecto.

1. _¿En palabras simples, qué es lo que realiza el interpolador entre cada par de *frames*?. **Hint: Un análisis frame a frame puede ayudar a responder esto.**_

2. _¿De qué manera se pueden mejorar los métodos estudiados para aumentar el framerate de un video?. **Hint: ¿Es adecuado considerar el conjunto de pixeles como independientes entre sí?**_

# Referencias
[1] Stock footage provided by Videvo, downloaded from https://www.videvo.net

[2] Barycentric Lagrange Interpolation, https://people.maths.ox.ac.uk/trefethen/barycentric.pdf